In [1]:
!pip install transformers
!pip install datasets

In [2]:
from google.colab import drive
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [3]:
!unzip /content/gdrive/MyDrive/com3025-text-challenge.zip

unzip:  cannot find or open /content/gdrive/MyDrive/com3025-text-challenge.zip, /content/gdrive/MyDrive/com3025-text-challenge.zip.zip or /content/gdrive/MyDrive/com3025-text-challenge.zip.ZIP.


In [3]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

from sklearn.preprocessing import MinMaxScaler
from sklearn.linear_model import LinearRegression
plt.style.use("ggplot")

#from keras.models import Sequential
#from keras.layers import LSTM, Dense, Dropout, Bidirectional

import requests
import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from transformers import BertTokenizer, BertForNextSentencePrediction, AutoTokenizer
import torch

In [4]:
model_checkpoint = 'bert-base-uncased'

In [5]:
tokenizer = BertTokenizer.from_pretrained(model_checkpoint)
model = BertForNextSentencePrediction.from_pretrained(model_checkpoint)

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForNextSentencePrediction: ['cls.predictions.transform.dense.weight', 'cls.predictions.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.LayerNorm.weight']
- This IS expected if you are initializing BertForNextSentencePrediction from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForNextSentencePrediction from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [6]:
train_path = "/content/gdrive/MyDrive/AI Year 3/Challenges/Week 4 and 5/training_data.csv"
test_path = "/content/gdrive/MyDrive/AI Year 3/Challenges/Week 4 and 5/test_data.csv"

train_data = pd.read_csv(train_path)
test_data = pd.read_csv(test_path)

In [7]:
train_data.head()

,storyid,sentence1,sentence2,sentence3,sentence4,sentence5
0,8bbe6d11-1e2e-413c-bf81-eaea05f4f1bd,David noticed he had put on a lot of weight re...,He examined his habits to try and figure out t...,He realized he'd been eating too much fast foo...,He stopped going to burger places and started ...,"After a few weeks, he started to feel much bet..."
1,0beabab2-fb49-460e-a6e6-f35a202e3348,Tom had a very short temper.,One day a guest made him very angry.,He punched a hole in the wall of his house.,Tom's guest became afraid and left quickly.,Tom sat on his couch filled with regret about ...
2,87da1a22-df0b-410c-b186-439700b70ba6,Marcus needed clothing for a business casual e...,All of his clothes were either too formal or t...,He decided to buy a pair of khakis.,The pair he bought fit him perfectly.,Marcus was happy to have the right clothes for...
3,2d16bcd6-692a-4fc0-8e7c-4a6f81d9efa9,Bobby thought Bill should buy a trailer and ha...,Bill thought a truck would be better for what ...,Bobby pointed out two vehicles were much more ...,Bill was set in his ways with conventional thi...,He ended up buying the truck he wanted despite...
4,c71bb23b-7731-4233-8298-76ba6886cee1,John was a pastor with a very bad memory.,He tried to memorize his sermons many days in ...,He decided to learn to sing to overcome his ha...,He then made all his sermons into music and sa...,His congregation was delighted and so was he.


In [8]:
train_data['text'] = ""
for index, row in train_data.iterrows():
  textls = []
  for i in row[1:]:
    textls.append(i)
  textls = ''.join(textls)
  train_data.at[index, 'text'] = textls
    


In [9]:
bag = []

for column in train_data:
    if column != "storyid":
        for row in train_data[column]:
            bag.append(row)
            #print(row)
    else:
        continue

In [10]:
train_data.head()

,storyid,sentence1,sentence2,sentence3,sentence4,sentence5,text
0,8bbe6d11-1e2e-413c-bf81-eaea05f4f1bd,David noticed he had put on a lot of weight re...,He examined his habits to try and figure out t...,He realized he'd been eating too much fast foo...,He stopped going to burger places and started ...,"After a few weeks, he started to feel much bet...",David noticed he had put on a lot of weight re...
1,0beabab2-fb49-460e-a6e6-f35a202e3348,Tom had a very short temper.,One day a guest made him very angry.,He punched a hole in the wall of his house.,Tom's guest became afraid and left quickly.,Tom sat on his couch filled with regret about ...,Tom had a very short temper.One day a guest ma...
2,87da1a22-df0b-410c-b186-439700b70ba6,Marcus needed clothing for a business casual e...,All of his clothes were either too formal or t...,He decided to buy a pair of khakis.,The pair he bought fit him perfectly.,Marcus was happy to have the right clothes for...,Marcus needed clothing for a business casual e...
3,2d16bcd6-692a-4fc0-8e7c-4a6f81d9efa9,Bobby thought Bill should buy a trailer and ha...,Bill thought a truck would be better for what ...,Bobby pointed out two vehicles were much more ...,Bill was set in his ways with conventional thi...,He ended up buying the truck he wanted despite...,Bobby thought Bill should buy a trailer and ha...
4,c71bb23b-7731-4233-8298-76ba6886cee1,John was a pastor with a very bad memory.,He tried to memorize his sermons many days in ...,He decided to learn to sing to overcome his ha...,He then made all his sermons into music and sa...,His congregation was delighted and so was he.,John was a pastor with a very bad memory.He tr...


In [11]:
tokenize_words = train_data['text'].apply(str)
tokenize_words = tokenize_words.to_string()

In [12]:
bag_size = len(bag)
bag_size

315990

In [19]:
import random

sentence_a = []
sentence_b = []
label = []

for index, row in train_data.iterrows():
#for paragraph in text:
  
    num_sentences = 5
    if num_sentences > 1:
        start = random.randint(1, num_sentences-2)
        # 50/50 whether is IsNextSentence or NotNextSentence
        if random.random() >= 0.5:
            # this is IsNextSentence
            sentence_a.append(row['sentence'+str(start)])
            sentence_b.append(row['sentence'+str(start+1)])
            label.append(0)
        else:
            index = random.randint(0, bag_size-1)
            # this is NotNextSentence
            sentence_a.append(row['sentence'+str(start)])
            sentence_b.append(bag[index])
            label.append(1)

In [20]:
i = 0
while i < 5:
  print(sentence_a[i])
  print(sentence_b[i])
  print(label[i])
  i += 1

He examined his habits to try and figure out the reason.
He installed the clean plug and soon was mowing the lawn.
1
One day a guest made him very angry.
He punched a hole in the wall of his house.
0
All of his clothes were either too formal or too casual.
He decided to buy a pair of khakis.
0
Bill thought a truck would be better for what he needed.
She had worked very hard throughout high school.
1
John was a pastor with a very bad memory.
He tried to memorize his sermons many days in advance but to no avail.
0


In [72]:
t_data=test_data

In [60]:
#from transformers import TFAutoModelForMultipleChoice

model = BertForNextSentencePrediction.from_pretrained(model_checkpoint)

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForNextSentencePrediction: ['cls.predictions.transform.dense.weight', 'cls.predictions.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.LayerNorm.weight']
- This IS expected if you are initializing BertForNextSentencePrediction from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForNextSentencePrediction from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [54]:
test_data.keys()

Index(['InputStoryid', 'InputSentence1', 'InputSentence2', 'InputSentence3',
       'InputSentence4', 'RandomFifthSentenceQuiz1',
       'RandomFifthSentenceQuiz2'],
      dtype='object')

In [57]:
storyid = t_data['InputStoryid'].values
sentences4 = t_data['InputSentence4'].values
sentences5 = t_data['RandomFifthSentenceQuiz1'].values
sentences6 = t_data['RandomFifthSentenceQuiz2'].values

sentences = []

for i in range(len(t_data)):
    text = []
    text.append(sentences4[i])
    text.append(sentences5[i])
    text.append(sentences6[i])
    text.append(storyid[i])
    sentences.append(text)

In [61]:
def loss(in1, in2):
    a = in1
    b = in2
    
    inputs = tokenizer(a, b, return_tensors='pt')
    
    labels = torch.LongTensor([0])
    
    outputs = model(**inputs, labels = labels)
    return [torch.argmax(outputs.logits), outputs.loss.item(), in1, in2]

In [63]:
list1 = []
list2 = []

for i in range(0,len(test_data)):
    list1.append(loss(sentences[i][0], sentences[i][1])[1])
    list2.append(loss(sentences[i][0], sentences[i][2])[1])

In [65]:
answerList = []
i = 0
while i < len(test_data):
    if list1[i] < list2[i]:
        answerList.append(0)
    else:
        answerList.append(1)
    i +=1

In [66]:
import csv

file_name="example_submission.csv"
with open(file_name, "w+", newline = "") as f:
    writer = csv.writer(f)
    writer.writerow(['InputStoryid', 'AnswerRightEnding'])
    for i in range(0,len(answerList)):
        writer.writerow([(sentences[i][3]), (answerList[i])])

In [70]:
from google.colab import files

In [71]:
files.download(file_name)

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [14]:
from dataclasses import dataclass
from transformers.tokenization_utils_base import PreTrainedTokenizerBase, PaddingStrategy
from typing import Optional, Union
import tensorflow as tf


@dataclass
class DataCollatorForMultipleChoice:
    """
    Data collator that will dynamically pad the inputs for multiple choice received.
    """

    tokenizer: PreTrainedTokenizerBase
    padding: Union[bool, str, PaddingStrategy] = True
    max_length: Optional[int] = None
    pad_to_multiple_of: Optional[int] = None

    def __call__(self, features):
        label_name = "label" if "label" in features[0].keys() else "labels"
        labels = [feature.pop(label_name) for feature in features]
        batch_size = len(features)
        num_choices = len(features[0]["input_ids"])
        flattened_features = [
            [{k: v[i] for k, v in feature.items()} for i in range(num_choices)] for feature in features
        ]
        flattened_features = sum(flattened_features, [])

        batch = self.tokenizer.pad(
            flattened_features,
            padding=self.padding,
            max_length=self.max_length,
            pad_to_multiple_of=self.pad_to_multiple_of,
            return_tensors="tf",
        )

        batch = {k: tf.reshape(v, (batch_size, num_choices, -1)) for k, v in batch.items()}
        batch["labels"] = tf.convert_to_tensor(labels, dtype=tf.int64)
        return batch

In [16]:
#data_collator = DataCollatorForMultipleChoice(tokenizer=tokenizer)
##tf_train_set = dataset_train.to_tf_dataset(
 #   columns=['storyid', 'sentence1', 'sentence2', 'sentence3', 'sentence4',
 #      'sentence5', 'text'],
 #   label_cols=["labels"],
 ##   shuffle=True,
  #  batch_size=batch_size,
  #  collate_fn=data_collator,
#)

#tf_validation_set = dataset_test.to_tf_dataset(
#    columns=['InputStoryid', 'InputSentence1', 'InputSentence2', 'InputSentence3',
#       'InputSentence4', 'RandomFifthSentenceQuiz1',
#       'RandomFifthSentenceQuiz2'],
#    label_cols=["labels"],
#    shuffle=False,
#    batch_size=batch_size,
#    collate_fn=data_collator,
#)

AttributeError: ignored

In [28]:
from datasets import load_dataset

swag = load_dataset("swag", "regular")

ending_names = ["ending0", "ending1", "ending2", "ending3"]


def preprocess_function(examples):
    first_sentences = [[context] * 4 for context in examples["sent1"]]
    question_headers = examples["sent2"]
    second_sentences = [
        [f"{header} {examples[end][i]}" for end in ending_names] for i, header in enumerate(question_headers)
    ]

    first_sentences = sum(first_sentences, [])
    second_sentences = sum(second_sentences, [])

    tokenized_examples = tokenizer(first_sentences, second_sentences, truncation=True)
    return {k: [v[i : i + 4] for i in range(0, len(v), 4)] for k, v in tokenized_examples.items()}

  
tokenized_swag = swag.map(preprocess_function, batched=True)

Reusing dataset swag (/root/.cache/huggingface/datasets/swag/regular/0.0.0/9640de08cdba6a1469ed3834fcab4b8ad8e38caf5d1ba5e7436d8b1fd067ad4c)


  0%|          | 0/3 [00:00<?, ?it/s]

Loading cached processed dataset at /root/.cache/huggingface/datasets/swag/regular/0.0.0/9640de08cdba6a1469ed3834fcab4b8ad8e38caf5d1ba5e7436d8b1fd067ad4c/cache-58a165efbf345989.arrow
Loading cached processed dataset at /root/.cache/huggingface/datasets/swag/regular/0.0.0/9640de08cdba6a1469ed3834fcab4b8ad8e38caf5d1ba5e7436d8b1fd067ad4c/cache-d5b0c84c885c6971.arrow
Loading cached processed dataset at /root/.cache/huggingface/datasets/swag/regular/0.0.0/9640de08cdba6a1469ed3834fcab4b8ad8e38caf5d1ba5e7436d8b1fd067ad4c/cache-d80af2ebcacf623b.arrow


In [29]:
tokenized_swag['train']

Dataset({
    features: ['video-id', 'fold-ind', 'startphrase', 'sent1', 'sent2', 'gold-source', 'ending0', 'ending1', 'ending2', 'ending3', 'label', 'input_ids', 'token_type_ids', 'attention_mask'],
    num_rows: 73546
})

In [31]:
dataset_train1 = train_data.to_xarray()

AttributeError: ignored

In [34]:
inputs

{'input_ids': tensor([[ 101, 2002, 8920,  ...,    0,    0,    0],
        [ 101, 2028, 2154,  ...,    0,    0,    0],
        [ 101, 2035, 1997,  ...,    0,    0,    0],
        ...,
        [ 101, 2096, 2551,  ...,    0,    0,    0],
        [ 101, 6175, 4122,  ...,    0,    0,    0],
        [ 101, 1996, 2611,  ...,    0,    0,    0]]), 'token_type_ids': tensor([[0, 0, 0,  ..., 0, 0, 0],
        [0, 0, 0,  ..., 0, 0, 0],
        [0, 0, 0,  ..., 0, 0, 0],
        ...,
        [0, 0, 0,  ..., 0, 0, 0],
        [0, 0, 0,  ..., 0, 0, 0],
        [0, 0, 0,  ..., 0, 0, 0]]), 'attention_mask': tensor([[1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0],
        ...,
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0]])}

In [35]:
inputs.token_type_ids[0]

tensor([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
        1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,

In [36]:
inputs['labels'] = torch.LongTensor([label]).T

In [37]:
inputs.labels[:10]

tensor([[1],
        [0],
        [0],
        [1],
        [0],
        [0],
        [0],
        [1],
        [0],
        [1]])

In [38]:
class StoryDataset(torch.utils.data.Dataset):
    def __init__(self, encodings):
        self.encodings = encodings
    def __getitem__(self, idx):
        return {key: val[idx].clone().detach() for key, val in self.encodings.items()}
    def __len__(self):
        return len(self.encodings.input_ids)

In [39]:
from transformers import default_data_collator
data_collator = default_data_collator

In [40]:
dataset = StoryDataset(inputs)

In [47]:
dataset[0].keys()

dict_keys(['input_ids', 'token_type_ids', 'attention_mask', 'labels'])

In [44]:
from transformers import create_optimizer

batch_size = 16
num_train_epochs = 2
total_train_steps = (len(train_data) // batch_size) * num_train_epochs
optimizer, schedule = create_optimizer(init_lr=5e-5, num_warmup_steps=0, num_train_steps=total_train_steps)

In [45]:
model = TFAutoModelForMultipleChoice.from_pretrained("bert-base-uncased")


All model checkpoint layers were used when initializing TFBertForMultipleChoice.

Some layers of TFBertForMultipleChoice were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [46]:
model.compile(
    optimizer=optimizer,
    loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
)

In [49]:
model.fit(dataset, epochs=2)

ValueError: ignored

In [ ]:
loader = torch.utils.data.DataLoader(dataset, batch_size=8, shuffle=True)

In [ ]:
torch.cuda.empty_cache()

In [ ]:
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
# and move our model over to the selected device
model = model.to(device)

In [ ]:
from transformers import AdamW

# activate training mode
model.train()
# initialize optimizer
optim = torch.optim.AdamW(model.parameters(), lr=5e-6)

In [ ]:
#torch.cuda.empty_cache()
from tqdm import tqdm  # for our progress bar


epochs = 2

for epoch in range(epochs):
    # setup loop with TQDM and dataloader
    loop = tqdm(loader, leave=True)
    for batch in loop:
        # initialize calculated gradients (from prev step)
        optim.zero_grad()
        # pull all tensor batches required for training
        input_ids = batch['input_ids'].to(device)
        attention_mask = batch['attention_mask'].to(device)
        token_type_ids = batch['token_type_ids'].to(device)
        labels = batch['labels'].to(device)
        # process
        outputs = model(input_ids, attention_mask=attention_mask,
                        token_type_ids=token_type_ids,
                        labels=labels)
        # extract loss
        loss = outputs.loss
        score = outputs.logits
        # calculate loss for every parameter that needs grad update
        loss.backward()
        # update parameters
        optim.step()
        # print relevant info to progress bar
        loop.set_description(f'Epoch {epoch}')
        loop.set_postfix(loss=loss.item())
        #loop.set_postfix(score=score.item())

In [ ]:

len(test_data)

test_data.head()


test_data['input_text'] = ""
for index, row in test_data.iterrows():
  textls = []
  for i in row[1:]:
    textls.append(i)
  textls = ''.join(textls)
  test_data.at[index, 'input_text'] = textls
#for i in range(0,len(test_data)):
#    list1.append(loss_calc(sentences[i][0], sentences[i][1])[1])
#    list2.append(loss_calc(sentences[i][0], sentences[i][2])[1])

In [ ]:
test_data.head()

In [ ]:

for index, row in test_data.iterrows():
  prompt = test_data.at[index,'input_text']
  next_sentence1 = test_data.at[index, 'RandomFifthSentenceQuiz1']
  next_sentence2 = test_data.at[index, 'RandomFifthSentenceQuiz2']
  encoding1 = tokenizer(prompt, next_sentence1, return_tensors='pt')
  encoding2 = tokenizer(prompt, next_sentence2, return_tensors='pt')

  loss1, logits1 = model(**encoding1, next_sentence_label=torch.LongTensor([1]))
  loss2, logits2 = model(**encoding2, next_sentence_label=torch.LongTensor([1]))

  if logits1 < logits2:
    test_data.at[index, 'correct_option'] = 1
  else:
    test_data.at[index,'correct_option'] = 0


In [ ]:
correct_option = test_data['correct_option']
input_id = test_data['InputStoryid']
filepath = "/content/drive/MyDrive/AI Year 3/Challenges/Week 4 and 5/submission.csv"


data = {
  "InputStoryid": input_id,
  "AnswerRightEnding": correct_option
}

submission_df = pd.DataFrame(data)

submission_df.to_csv(filepath) 

In [ ]:
torch.argmax(outputs.logits)

In [ ]:
train_data.keys()

In [ ]:
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint, use_fast=True)

In [ ]:
def tokenize_function(examples):
    return tokenizer(examples)

In [ ]:
tokenizer(sentences)

In [ ]:

tokenized_datasets = datasets.map(tokenize_function, batched=True, num_proc=4, remove_columns=["text"])